In [1]:
import sys
import importlib
import os

import files
importlib.reload(files)

import fonctions
importlib.reload(fonctions)

from files import *
from fonctions import *

In [2]:
consommation_chauffage_toulouse = extract_and_concat_consommation(toulouse, column_index=4, prefix="consommation_heat_")
consommation_chauffage_zurich = extract_and_concat_consommation(zurich, column_index=4, prefix="consommation_heat_")
consommation_chauffage_seville = extract_and_concat_consommation(seville, column_index=4, prefix="consommation_heat_")
consommation_climatisation_toulouse = extract_and_concat_consommation(toulouse, column_index=5, prefix="consommation_cool_")
consommation_climatisation_zurich = extract_and_concat_consommation(zurich, column_index=5, prefix="consommation_cool_")
consommation_climatisation_seville = extract_and_concat_consommation(seville, column_index=5, prefix="consommation_cool_")


city_groups = {
    "toulouse": toulouse_meteo,
    "zurich": zurich_meteo,
    "seville": seville_meteo
}

prefix_column_map = {
    "Text_": 1,
    "Hum_": 3,
    "Wind_": 4,
    "Solar_": 5,
    "Ground_": 10
}

combined_data = extract_and_combine_all(city_groups, prefix_column_map)

Text_combined_toulouse = combined_data.get('Text_combined_toulouse')
Hum_combined_toulouse = combined_data.get('Hum_combined_toulouse')


In [3]:


# --- Fonction pour calculer les métriques ---
def compute_metrics(predictions, targets):
    mse = torch.mean((predictions - targets) ** 2)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(predictions - targets))
    ss_total = torch.sum((targets - torch.mean(targets)) ** 2)
    ss_residual = torch.sum((targets - predictions) ** 2)
    r2 = 1 - (ss_residual / ss_total)
    cvrmse = (rmse / torch.mean(targets)) * 100
    return mae, rmse, mse, r2, cvrmse

In [8]:
import torch
import torch.nn as nn
import math

# Encodage positionnel pour donner une notion du temps au modèle
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [79]:
class TimeSeriesTransformerWithDecoder(nn.Module):
    def __init__(self, num_features=3, d_model=64, nhead=4, num_layers=2, dim_feedforward=128, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.input_proj = nn.Linear(num_features, d_model)
        self.target_proj = nn.Linear(1, d_model)

        self.pos_encoder = PositionalEncoding(d_model)
        self.pos_decoder = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.output_layer = nn.Linear(d_model, 1)

    def generate_square_subsequent_mask(self, sz):
        # Masque pour empêcher de voir le futur
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

    def forward(self, src, tgt, use_mask=False):
        # src: (batch, 24, num_features) / tgt: (batch, 24)
        src = self.input_proj(src)           # (batch, 24, d_model)
        src = self.pos_encoder(src)

        memory = self.encoder(src)

        # Préparer tgt (conso précédente décalée d’un pas)
        tgt = tgt.unsqueeze(-1)              # (batch, 24, 1)
        tgt = self.target_proj(tgt)          # (batch, 24, d_model)
        tgt = self.pos_decoder(tgt)

        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1)).to(tgt.device) if use_mask else None

        output = self.decoder(tgt, memory, tgt_mask=tgt_mask)
        output = self.output_layer(output).squeeze(-1)  # (batch, 24)
        return output


In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

class TimeSeriesDataset(Dataset):
    def __init__(self, df, input_blocks, target_block):
        """
        df : DataFrame avec toutes les variables concaténées par blocs de 24h
        input_blocks : liste de tuples (col_start, col_end, day_offset)
                       où day_offset = 0 pour aujourd’hui, -1 pour hier, etc.
        target_block : tuple (col_start, col_end) pour la target (conso aujourd’hui)
        """
        self.x = []
        self.y = []

        for i in range(1, len(df)):  # Commencer à 1 pour avoir accès à hier
            input_seq = []

            for start, end, offset in input_blocks:
                row_index = i + offset

                # Vérification que row_index est valide
                if 0 <= row_index < len(df):
                    values = df.iloc[row_index, start:end].values  # shape: (24,)
                else:
                    values = np.zeros(end - start)  # Utiliser des zéros si l'index est hors limites
                input_seq.append(values)

            # shape finale: (24, num_features)
            input_seq = np.stack(input_seq, axis=1)
            self.x.append(input_seq)

            # Target: consommation aujourd’hui
            target = df.iloc[i, target_block[0]:target_block[1]].values
            self.y.append(target)

        self.x = torch.tensor(self.x, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


def prepare_data(df_scaled, n_features):

    input_blocks = []

    for i in range(n_features):
        start = i * 24
        end = (i + 1) * 24

        # Aujourd’hui
        input_blocks.append((start, end, 0))
        # Hier
        input_blocks.append((start, end, -1))

    # Ajouter la consommation d’hier (toujours la dernière feature)
    conso_start = n_features * 24
    conso_end = conso_start + 24
    input_blocks.append((conso_start, conso_end, -1))
    target_block = (df_scaled.shape[1] - 24, df_scaled.shape[1])  
    print(input_blocks)
    print(target_block)
    df_trainval, df_test = train_test_split(df_scaled, test_size=0.2, shuffle=False)
    df_train, df_val = train_test_split(df_trainval, test_size=0.1, shuffle=False)
    
    train_dataset = TimeSeriesDataset(df_train.reset_index(drop=True), input_blocks, target_block)
    val_dataset = TimeSeriesDataset(df_val.reset_index(drop=True), input_blocks, target_block)
    test_dataset = TimeSeriesDataset(df_test.reset_index(drop=True), input_blocks, target_block)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    return train_loader, val_loader, test_loader


In [27]:
df2 = Text_combined_toulouse.copy()
df2=pd.concat([df2,consommation_chauffage_toulouse],axis=1).reset_index(drop=True)
df2.columns = [f"col_{i}" for i in range(df2.shape[1])]

target_columns = df2.columns[-24:]  
input_columns = df2.columns[:-24]

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = pd.DataFrame(scaler_X.fit_transform(df2[input_columns]), columns=input_columns)
y_scaled = pd.DataFrame(scaler_y.fit_transform(df2[target_columns]), columns=target_columns)
df_scaled = pd.concat([X_scaled, y_scaled], axis=1)


train_loader, val_loader, test_loader = prepare_data(df_scaled,1)


[(0, 24, 0), (0, 24, -1), (24, 48, -1)]
(24, 48)


In [81]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# --- Fonction pour calculer les métriques ---
def compute_metrics(predictions, targets):
    mse = torch.mean((predictions - targets) ** 2)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(predictions - targets))
    return mae, rmse, mse

# --- Fonction pour calculer R2 et CVRMSE (utilisé seulement lors du test/validation) ---
def compute_test_metrics(predictions, targets):
    mse = torch.mean((predictions - targets) ** 2)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(predictions - targets))
    ss_total = torch.sum((targets - torch.mean(targets)) ** 2)
    ss_residual = torch.sum((targets - predictions) ** 2)
    r2 = 1 - (ss_residual / (ss_total + 1e-8))
    cvrmse = (rmse / (torch.mean(targets)+ 1e-8)) * 100
    return mae, rmse, mse, r2, cvrmse

# Initialisation du modèle
model = TimeSeriesTransformerWithDecoder(
    num_features=3,              # 3 features par pas de temps : température, statut, cluster
    d_model=64,
    nhead=4,
    num_layers=2,
    dim_feedforward=128,
    dropout=0.1
)
model.to(device)

# Optimiseur et fonction de perte
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Boucle d'entraînement
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_mae = 0.0
    total_rmse = 0.0
    total_mse = 0.0

    for batch_idx, (src, tgt) in enumerate(train_loader):
        src, tgt = src.to(device), tgt.to(device)

       
        #y_minus_1 = src[:, -1, -1].unsqueeze(1)    
        #tgt_input = torch.cat([y_minus_1, tgt[:, :-1]], dim=1) 
        #tgt_target = tgt 

        
        batch_size, pred_len = tgt.size()  # pred_len = 24
        tgt_target = tgt 
        decoder_input = src[:, -1, -1].unsqueeze(1)  # y_{t-1}, [batch, 1]

        predictions = []
        for t in range(pred_len):
            out = model(src, decoder_input, use_mask=False)         # out: [batch, t+1]
            next_val = out[:, -1:]                  # dernière valeur prédite [batch, 1]
            predictions.append(next_val)
            decoder_input = torch.cat([decoder_input, next_val], dim=1)  # accumulate

        output = torch.cat(predictions, dim=1)  # [batch, 24]

        
        
        loss = criterion(output, tgt_target)
        mae, rmse, mse = compute_metrics(output, tgt_target)

        total_loss += loss.item()
        total_mae += mae.item()
        total_rmse += rmse.item()
        total_mse += mse.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Affichage des métriques d'entraînement
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {total_loss / len(train_loader):.4f} | "
          f"MAE: {total_mae / len(train_loader):.4f} | RMSE: {total_rmse / len(train_loader):.4f} | "
          f"MSE: {total_mse / len(train_loader):.4f}")

        # Validation
    if (epoch + 1) % 5 == 0:
        model.eval()
        val_loss = 0.0
        val_mae = 0.0
        val_rmse = 0.0
        val_mse = 0.0
        val_r2 = 0.0
        val_cvrmse = 0.0
        with torch.no_grad():
            for batch_idx, (src, tgt) in enumerate(val_loader):
                src, tgt = src.to(device), tgt.to(device)
    
                #y_minus_1 = src[:, -1, -1].unsqueeze(1) 
                #tgt_input = torch.cat([y_minus_1, tgt[:, :-1]], dim=1) 
                #tgt_target = tgt 
                #output = model(src, tgt_input)
                batch_size, pred_len = tgt.size()  # pred_len = 24
                tgt_target = tgt
                decoder_input = src[:, -1, -1].unsqueeze(1)  # y_{t-1}, [batch, 1]

                predictions = []
                for t in range(pred_len):
                    out = model(src, decoder_input, use_mask=False)         # out: [batch, t+1]
                    next_val = out[:, -1:]                  # dernière valeur prédite [batch, 1]
                    predictions.append(next_val)
                    decoder_input = torch.cat([decoder_input, next_val], dim=1)  # accumulate

                output = torch.cat(predictions, dim=1)  # [batch, 24]
        
                
    
                # Inverser la normalisation pour R2 et CVRMSE
                output_unscaled = scaler_y.inverse_transform(output.cpu().numpy())  # Convertir en numpy
                tgt_unscaled = scaler_y.inverse_transform(tgt_target.cpu().numpy())
    
                # Calcul des métriques standards (MAE, RMSE, MSE) sur les valeurs normalisées
                loss = criterion(output, tgt_target)
                mae, rmse, mse = compute_metrics(output, tgt_target)
    
                # Calcul de R2 et CVRMSE après inverse de la normalisation
                # Inverser la normalisation pour R2 et CVRMSE
                ss_total = torch.sum((torch.tensor(tgt_unscaled).to(device) - torch.mean(torch.tensor(tgt_unscaled).to(device))) ** 2)
                ss_residual = torch.sum((torch.tensor(tgt_unscaled).to(device) - torch.tensor(output_unscaled).to(device)) ** 2)
                r2 = 1 - (ss_residual / (ss_total ))
                rmse_unscaled = torch.sqrt(torch.mean((torch.tensor(output_unscaled).to(device) - torch.tensor(tgt_unscaled).to(device)) ** 2))
                cvrmse = (rmse_unscaled / (torch.mean(torch.tensor(tgt_unscaled).to(device)) )) * 100
    
                # Mise à jour des métriques de validation
                val_loss += loss.item()
                val_mae += mae.item()
                val_rmse += rmse.item()
                val_mse += mse.item()
                val_r2 += r2.item()
                val_cvrmse += cvrmse.item()
    
        # Affichage des métriques de validation
        print(f"→ Validation Loss: {val_loss / len(val_loader):.4f} | "
              f"MAE: {val_mae / len(val_loader):.4f} | RMSE: {val_rmse / len(val_loader):.4f} | "
              f"MSE: {val_mse / len(val_loader):.4f} | R2: {val_r2 / len(val_loader):.4f} | "
              f"CVRMSE: {val_cvrmse / len(val_loader):.4f}")


Epoch [1/10] | Train Loss: 0.1175 | MAE: 0.2595 | RMSE: 0.3193 | MSE: 0.1175
Epoch [2/10] | Train Loss: 0.0291 | MAE: 0.1289 | RMSE: 0.1688 | MSE: 0.0291
Epoch [3/10] | Train Loss: 0.0191 | MAE: 0.1016 | RMSE: 0.1371 | MSE: 0.0191
Epoch [4/10] | Train Loss: 0.0163 | MAE: 0.0933 | RMSE: 0.1264 | MSE: 0.0163
Epoch [5/10] | Train Loss: 0.0142 | MAE: 0.0852 | RMSE: 0.1176 | MSE: 0.0142
→ Validation Loss: 0.0218 | MAE: 0.1165 | RMSE: 0.1459 | MSE: 0.0218 | R2: 0.6213 | CVRMSE: 58.1218
Epoch [6/10] | Train Loss: 0.0128 | MAE: 0.0797 | RMSE: 0.1125 | MSE: 0.0128
Epoch [7/10] | Train Loss: 0.0114 | MAE: 0.0743 | RMSE: 0.1060 | MSE: 0.0114
Epoch [8/10] | Train Loss: 0.0112 | MAE: 0.0740 | RMSE: 0.1046 | MSE: 0.0112
Epoch [9/10] | Train Loss: 0.0105 | MAE: 0.0692 | RMSE: 0.1011 | MSE: 0.0105
Epoch [10/10] | Train Loss: 0.0102 | MAE: 0.0705 | RMSE: 0.1004 | MSE: 0.0102
→ Validation Loss: 0.0181 | MAE: 0.1058 | RMSE: 0.1330 | MSE: 0.0181 | R2: 0.6880 | CVRMSE: 52.7623


In [82]:
# --- Phase de test ---
model.eval()
test_loss = 0.0
test_mae = 0.0
test_rmse = 0.0
test_mse = 0.0
test_r2 = 0.0
test_cvrmse = 0.0

with torch.no_grad():
    for batch_idx, (src, tgt) in enumerate(test_loader):
        src, tgt = src.to(device), tgt.to(device)
        tgt_target = tgt  # [batch, 24]
        batch_size, pred_len = tgt.size()

        decoder_input = src[:, -1, -1].unsqueeze(1)  # y_{t-1}, shape: [batch, 1]
        predictions = []

        for t in range(pred_len):
            out = model(src, decoder_input, use_mask=False)
            next_val = out[:, -1:]  # [batch, 1]
            predictions.append(next_val)
            decoder_input = torch.cat([decoder_input, next_val], dim=1)

        output = torch.cat(predictions, dim=1)  # [batch, 24]

        # Inverse transform pour métriques réelles
        output_unscaled = scaler_y.inverse_transform(output.cpu().numpy())
        tgt_unscaled = scaler_y.inverse_transform(tgt_target.cpu().numpy())

        # Conversion vers tensors
        output_unscaled_tensor = torch.tensor(output_unscaled).to(device)
        tgt_unscaled_tensor = torch.tensor(tgt_unscaled).to(device)

        # Calcul des métriques normalisées (pour suivi)
        loss = criterion(output, tgt_target)
        mae, rmse, mse, r2, cvrmse = compute_test_metrics(output_unscaled_tensor, tgt_unscaled_tensor)

        test_loss += loss.item()
        test_mae += mae.item()
        test_rmse += rmse.item()
        test_mse += mse.item()
        test_r2 += r2.item()
        test_cvrmse += cvrmse.item()

print(f"\n=== TEST SET RESULTS ===")
print(f"Test Loss: {test_loss / len(test_loader):.4f} | "
      f"MAE: {test_mae / len(test_loader):.4f} | RMSE: {test_rmse / len(test_loader):.4f} | "
      f"MSE: {test_mse / len(test_loader):.4f} | R2: {test_r2 / len(test_loader):.4f} | "
      f"CVRMSE: {test_cvrmse / len(test_loader):.4f}")



=== TEST SET RESULTS ===
Test Loss: 0.0037 | MAE: 146.0478 | RMSE: 181.6176 | MSE: 77264.1401 | R2: -16859116392354.9043 | CVRMSE: 618380625232.9381
